# Trying standard QAOA

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
!export CUDA_VISIBLE_DEVICES=1,2,3

import cudaq
from cudaq import spin
import numpy as np
from scipy.optimize import minimize
import random
import copy

# ============================================================
# CUDA-Q QAOA Implementation for LABS
# ============================================================

def labs_hamiltonian(n: int) -> cudaq.SpinOperator:
    """Construct the LABS cost Hamiltonian."""
    hamiltonian = 0.0 * spin.i(0)
    
    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    hamiltonian += 2.0 * spin.z(ii) * spin.z(ii + tt) * spin.z(ii + kk) * spin.z(ii + kk + tt)
    
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                hamiltonian += spin.z(ii) * spin.z(jj)
    
    return hamiltonian

def get_labs_interactions_flat(n: int):
    """Pre-compute LABS interactions as flat lists."""
    fb_0, fb_1, fb_2, fb_3 = [], [], [], []
    
    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    fb_0.append(ii)
                    fb_1.append(ii + tt)
                    fb_2.append(ii + kk)
                    fb_3.append(ii + kk + tt)
    
    tb_0, tb_1 = [], []
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                tb_0.append(ii)
                tb_1.append(jj)
    
    return fb_0, fb_1, fb_2, fb_3, tb_0, tb_1

@cudaq.kernel
def labs_qaoa_kernel(
    n_qubits: int,
    p_layers: int,
    gamma: list[float], 
    beta: list[float],
    fb_0: list[int], fb_1: list[int], fb_2: list[int], fb_3: list[int],
    tb_0: list[int], tb_1: list[int],
    n_four_body: int,
    n_two_body: int
):
    """QAOA kernel for LABS problem."""
    qubits = cudaq.qvector(n_qubits)
    
    # Initial superposition
    h(qubits)
    
    # Apply p QAOA layers
    for layer in range(p_layers):
        # Phase separator: 4-body ZZZZ terms
        for idx in range(n_four_body):
            # Use list values directly as indices
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            rz(4.0 * gamma[layer], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
        
        # Phase separator: 2-body ZZ terms
        for idx in range(n_two_body):
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
            rz(2.0 * gamma[layer], qubits[tb_1[idx]])
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
        
        # Mixer: exp(-i * beta * sum_j X_j)
        for q_idx in range(n_qubits):
            rx(2.0 * beta[layer], qubits[q_idx])

# ============================================================
# Bridge: Convert QAOA results to MTS population format
# ============================================================

def bitstring_to_list(bitstring: str) -> list[int]:
    """Convert a bitstring like '10101' to a list of ints [1, 0, 1, 0, 1]."""
    return [int(bit) for bit in bitstring]

def get_top_k_bitstrings(counts, k_pop: int, n_bits: int) -> list[list[int]]:
    """
    Extract the top-k most frequently measured bitstrings from QAOA results.
    
    Args:
        counts: Sample results from cudaq.sample()
        k_pop: Number of bitstrings to extract for MTS population
        n_bits: Expected bitstring length (for padding if needed)
    
    Returns:
        List of lists: [[0, 1, 0, ...], [1, 1, 0, ...], ...]
    """
    # Sort by count (most frequent first)
    sorted_bitstrings = sorted(counts.items(), key=lambda x: -x[1])
    
    population = []
    for bitstring, count in sorted_bitstrings[:k_pop]:
        # Pad if necessary (CUDA-Q may omit leading zeros)
        padded = bitstring.zfill(n_bits)
        individual = bitstring_to_list(padded)
        population.append(individual)
    
    # If we don't have enough unique bitstrings, duplicate some
    while len(population) < k_pop:
        population.append(copy.deepcopy(random.choice(population)))
    
    return population

def run_qaoa_for_mts_seeding(n_qubits: int, k_pop: int, p_layers: int = 2, 
                             shots: int = 4096, optimize: bool = True):
    """
    Run QAOA and return top-k bitstrings formatted for MTS.
    
    Args:
        n_qubits: Problem size (N)
        k_pop: Population size for MTS
        p_layers: Number of QAOA layers
        shots: Number of measurement shots
        optimize: Whether to optimize QAOA parameters
    
    Returns:
        List of lists suitable for MTS initial_population
    """
    # Pre-compute interactions
    fb_0, fb_1, fb_2, fb_3, tb_0, tb_1 = get_labs_interactions_flat(n_qubits)
    n_four_body = len(fb_0)
    n_two_body = len(tb_0)
    
    print(f"  Problem size: {n_qubits} qubits")
    print(f"  4-body interactions: {n_four_body}")
    print(f"  2-body interactions: {n_two_body}")
    
    if optimize:
        # Build Hamiltonian for optimization
        hamiltonian = labs_hamiltonian(n_qubits)
        
        def cost_function(params):
            gamma_params = params[:p_layers].tolist()
            beta_params = params[p_layers:].tolist()
            return cudaq.observe(
                labs_qaoa_kernel, hamiltonian,
                n_qubits, p_layers,
                gamma_params, beta_params,
                fb_0, fb_1, fb_2, fb_3,
                tb_0, tb_1,
                n_four_body, n_two_body
            ).expectation()
        
        # Optimize parameters
        x0 = np.array([0.1 * (idx + 1) / p_layers for idx in range(p_layers)] +
                      [0.5 * (p_layers - idx) / p_layers for idx in range(p_layers)])
        
        print(f"  Optimizing QAOA parameters...")
        result = minimize(cost_function, x0, method='COBYLA',
                         options={'maxiter': 100, 'rhobeg': 0.3})
        
        optimal_gamma = result.x[:p_layers].tolist()
        optimal_beta = result.x[p_layers:].tolist()
        print(f"  Optimization complete. Energy: {result.fun:.4f}")
    else:
        # Use linear ramp schedule
        optimal_gamma = [0.1 * (idx + 1) / p_layers for idx in range(p_layers)]
        optimal_beta = [0.5 * (p_layers - idx) / p_layers for idx in range(p_layers)]
    
    # Sample from optimized circuit
    print(f"  Sampling {shots} shots...")
    counts = cudaq.sample(
        labs_qaoa_kernel,
        n_qubits, p_layers,
        optimal_gamma, optimal_beta,
        fb_0, fb_1, fb_2, fb_3,
        tb_0, tb_1,
        n_four_body, n_two_body,
        shots_count=shots
    )
    
    # Convert to MTS format
    population = get_top_k_bitstrings(counts, k_pop, n_qubits)
    
    return population

# ============================================================
# Your MTS Code (unchanged)
# ============================================================

def calculate_energy(s: list[int], N: int):
    if len(s) < N:
        s = [0] * (N - len(s)) + s
    sequence = [2*x - 1 for x in s] 
    energy = 0
    for k_val in range(1, N):
        c_k = 0
        for i_val in range(N - k_val):
            c_k += sequence[i_val] * sequence[i_val + k_val]
        energy += c_k**2
    return energy

def combine(s1: list[int], s2: list[int], N: int):
    if N > 1:
        cut_pt = random.randint(1, N - 1) 
        return s1[0:cut_pt] + s2[cut_pt:]
    else:
        return s1

def mutate(s: list[int], p: float, N: int):
    for i_val in range(N):
        if random.random() < p: 
            s[i_val] = s[i_val] ^ 1 

def tabu_search(N: int, s: list[int], max_iters=100, tabu_tenure=5):
    current_s = copy.deepcopy(s)
    best_s = copy.deepcopy(s)
    best_energy = calculate_energy(current_s, N)
    tabu_list = [] 
    
    for _ in range(max_iters):
        local_best_neighbor = None
        local_best_energy = float('inf')
        move_index = -1
        
        for i_val in range(N):
            current_s[i_val] ^= 1
            neighbor_energy = calculate_energy(current_s, N)
            
            is_tabu = i_val in tabu_list
            is_aspiration = neighbor_energy < best_energy
            
            if (not is_tabu) or is_aspiration:
                if neighbor_energy < local_best_energy:
                    local_best_energy = neighbor_energy
                    move_index = i_val
                    local_best_neighbor = copy.deepcopy(current_s)

            current_s[i_val] ^= 1
        
        if local_best_neighbor is not None:
            current_s = local_best_neighbor
            
            if local_best_energy < best_energy:
                best_energy = local_best_energy
                best_s = copy.deepcopy(current_s)
            
            tabu_list.append(move_index)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
    
    return best_s

def generate_random_bitstring(k_pop, N):
    population = []
    for _ in range(k_pop):
        bitstring = [random.randint(0, 1) for _ in range(N)]
        population.append(bitstring)
    return population

def MTS(N, k_pop, num_iterations, mutation_rate=0.05, population=None):
    if population is None or len(population) == 0:
        population = generate_random_bitstring(k_pop, N)
    
    best_solution = population[0]
    best_energy = calculate_energy(best_solution, N)

    for ind in population:
        e = calculate_energy(ind, N)
        if e < best_energy:
            best_energy = e
            best_solution = copy.deepcopy(ind)

    print(f"Initial Best Energy: {best_energy}")

    for it in range(num_iterations):
        child = []
        if it == 0:
            child = copy.deepcopy(random.choice(population))
        else:
            p1 = random.choice(population)
            p2 = random.choice(population)
            child = combine(p1, p2, N)

        mutate(child, mutation_rate, N)
        improved_child = tabu_search(N, child, max_iters=N*2) 
        improved_child_energy = calculate_energy(improved_child, N)

        if improved_child_energy < best_energy:
            print(f"Iteration {it}: New Best Energy Found: {improved_child_energy}")
            best_energy = improved_child_energy
            best_solution = copy.deepcopy(improved_child)
            replace_idx = random.randint(0, k_pop - 1)
            population[replace_idx] = improved_child

    return best_solution, best_energy

# ============================================================
# Main: Hybrid Quantum-Classical LABS Solver
# ============================================================

if __name__ == "__main__":
    N = 30
    k = 8
    mts_iterations = 50
    mutation_rate = 0.05

    print("=" * 60)
    print("HYBRID QUANTUM-CLASSICAL LABS SOLVER")
    print("=" * 60)

    # Step 1: Run QAOA to get quantum-seeded initial population
    print("\n[Step 1] Running QAOA to generate initial population...")
    initial_population = run_qaoa_for_mts_seeding(
        n_qubits=N, 
        k_pop=k, 
        p_layers=2,
        shots=4096,
        optimize=True
    )

    print(f"\nQuantum-seeded population ({k} individuals):")
    for idx, ind in enumerate(initial_population):
        energy = calculate_energy(ind, N)
        print(f"  Individual {idx}: {ind} -> Energy: {energy}")

    # Step 2: Run MTS with quantum-seeded population
    print(f"\n[Step 2] Running MTS with quantum-seeded population...")
    best_solution, best_energy = MTS(
        N=N, 
        k_pop=k, 
        num_iterations=mts_iterations, 
        mutation_rate=mutation_rate, 
        population=initial_population
    )

    print("\n" + "=" * 60)
    print("FINAL RESULTS")
    print("=" * 60)
    print(f"Best solution: {best_solution}")
    print(f"Best energy: {best_energy}")
    if best_energy > 0:
        print(f"Merit factor: {N**2 / (2 * best_energy):.4f}")
    else:
        print("Perfect solution found!")


HYBRID QUANTUM-CLASSICAL LABS SOLVER

[Step 1] Running QAOA to generate initial population...
  Problem size: 30 qubits
  4-body interactions: 1925
  2-body interactions: 210
  Optimizing QAOA parameters...


In [4]:
"""
HYBRID QAOA-SEEEDED MTS FOR LABS (Optimized for single A100)

Key changes vs your original:
1) Single-GPU: set CUDA_VISIBLE_DEVICES BEFORE importing cudaq.
2) Much lower VRAM + faster optimization: NO cudaq.observe() / Pauli-sum expectation.
   We optimize directly using shot-based sampling + classical LABS energy.
3) Optional term subsampling (huge speedup): use only a fraction of 4-body / 2-body terms
   when *seeding* MTS. This keeps the circuit much smaller but still biases toward good bitstrings.
4) Prints circuit stats: gate counts + serial depth estimate (based on your exact decomposition).
5) Faster MTS tabu search: incremental energy updates (no O(N^2) recompute per neighbor).

Notes:
- If you run this in a notebook, you MUST restart the kernel after changing CUDA_VISIBLE_DEVICES.
- For best speed, try fraction_four=0.25 (or even 0.1) for seeding; MTS does the heavy lifting anyway.
"""

# ------------------------------------------------------------
# 0) Single GPU selection MUST happen before importing cudaq
# ------------------------------------------------------------
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # <-- set to your A100's visible index

import random
import copy
import numpy as np
from scipy.optimize import minimize

import cudaq
from cudaq import spin

# ------------------------------------------------------------
# 1) LABS interactions (same structure you had, but only lists)
# ------------------------------------------------------------
def get_labs_interactions_flat(n: int):
    """Pre-compute LABS interactions as flat lists (4-body and 2-body)."""
    fb_0, fb_1, fb_2, fb_3 = [], [], [], []

    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    fb_0.append(ii)
                    fb_1.append(ii + tt)
                    fb_2.append(ii + kk)
                    fb_3.append(ii + kk + tt)

    tb_0, tb_1 = [], []
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                tb_0.append(ii)
                tb_1.append(jj)

    return fb_0, fb_1, fb_2, fb_3, tb_0, tb_1


def subsample_interactions(
    fb_0, fb_1, fb_2, fb_3,
    tb_0, tb_1,
    fraction_four: float = 1.0,
    fraction_two: float = 1.0,
    seed: int = 0
):
    """
    Subsample a fixed fraction of interactions to reduce gate count drastically.
    This is VERY useful for seeding MTS: smaller QAOA circuit, much faster.
    """
    rng = random.Random(seed)

    # 4-body
    n4 = len(fb_0)
    keep4 = max(1, int(round(fraction_four * n4))) if n4 > 0 else 0
    if keep4 < n4:
        idx4 = rng.sample(range(n4), keep4)
        idx4.sort()
        fb_0_s = [fb_0[i] for i in idx4]
        fb_1_s = [fb_1[i] for i in idx4]
        fb_2_s = [fb_2[i] for i in idx4]
        fb_3_s = [fb_3[i] for i in idx4]
    else:
        fb_0_s, fb_1_s, fb_2_s, fb_3_s = fb_0, fb_1, fb_2, fb_3

    # 2-body
    n2 = len(tb_0)
    keep2 = max(1, int(round(fraction_two * n2))) if n2 > 0 else 0
    if keep2 < n2:
        idx2 = rng.sample(range(n2), keep2)
        idx2.sort()
        tb_0_s = [tb_0[i] for i in idx2]
        tb_1_s = [tb_1[i] for i in idx2]
    else:
        tb_0_s, tb_1_s = tb_0, tb_1

    return fb_0_s, fb_1_s, fb_2_s, fb_3_s, tb_0_s, tb_1_s


# ------------------------------------------------------------
# 2) QAOA kernel (same decomposition you used)
# ------------------------------------------------------------
@cudaq.kernel
def labs_qaoa_kernel(
    n_qubits: int,
    p_layers: int,
    gamma: list[float],
    beta: list[float],
    fb_0: list[int], fb_1: list[int], fb_2: list[int], fb_3: list[int],
    tb_0: list[int], tb_1: list[int],
    n_four_body: int,
    n_two_body: int
):
    qubits = cudaq.qvector(n_qubits)

    # Initial superposition
    h(qubits)

    for layer in range(p_layers):
        # 4-body ZZZZ terms via parity chain + RZ + uncompute
        for idx in range(n_four_body):
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            rz(4.0 * gamma[layer], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])

        # 2-body ZZ terms
        for idx in range(n_two_body):
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
            rz(2.0 * gamma[layer], qubits[tb_1[idx]])
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])

        # Mixer
        for q_idx in range(n_qubits):
            rx(2.0 * beta[layer], qubits[q_idx])


# ------------------------------------------------------------
# 3) Circuit stats (gate counts + serial depth estimate)
# ------------------------------------------------------------
def qaoa_gate_stats(N: int, p: int, n_four: int, n_two: int):
    # Based exactly on your decomposition:
    # 4-body term: 6 CNOT + 1 RZ
    # 2-body term: 2 CNOT + 1 RZ
    # Mixer: N RX per layer (serialized in your loop)
    # Init: N H
    stats = {
        "H": N,
        "RX": p * N,
        "RZ": p * (n_four + n_two),
        "CNOT": p * (6 * n_four + 2 * n_two),
    }
    total_gates = stats["H"] + stats["RX"] + stats["RZ"] + stats["CNOT"]

    # Serial depth estimate (your literal loop order; no parallel scheduling):
    # Init H: depth ~1
    # Per 4-body: 7 ops (6 CNOT + 1 RZ)
    # Per 2-body: 3 ops (2 CNOT + 1 RZ)
    # Mixer: N ops (since you loop)
    depth_serial = 1 + p * (7 * n_four + 3 * n_two + N)
    return stats, total_gates, depth_serial


# ------------------------------------------------------------
# 4) LABS energy + fast incremental helpers (for tabu search)
# ------------------------------------------------------------
def bits_to_pm1(bits01: list[int]) -> list[int]:
    # 0->-1, 1->+1
    return [2 * b - 1 for b in bits01]


def compute_correlations_pm1(xpm1: list[int]):
    """
    C[k] = sum_{i=0}^{N-k-1} x[i]*x[i+k], for k=1..N-1.
    We'll store in a list C of length N, with C[0]=0 unused.
    """
    N = len(xpm1)
    C = [0] * N
    for k in range(1, N):
        s = 0
        for i in range(N - k):
            s += xpm1[i] * xpm1[i + k]
        C[k] = s
    return C


def energy_from_correlations(C: list[int]) -> int:
    # E = sum_{k=1}^{N-1} C[k]^2
    return sum(v * v for v in C[1:])


def calculate_energy(bits01: list[int], N: int) -> int:
    """Standalone LABS energy for a bitstring (0/1)."""
    if len(bits01) < N:
        bits01 = [0] * (N - len(bits01)) + bits01
    x = bits_to_pm1(bits01)
    C = compute_correlations_pm1(x)
    return energy_from_correlations(C)


def delta_energy_if_flip(xpm1: list[int], C: list[int], E: int, j: int) -> int:
    """
    Compute neighbor energy if we flip bit j (x[j] -> -x[j]) using O(N).
    Flip affects each C[k] by:
      ΔC[k] = -2*x[j]*x[j+k]  (if j+k < N)  +  -2*x[j-k]*x[j] (if j-k >= 0)
           = -2*x[j]*( (j+k<N? x[j+k]:0) + (j-k>=0? x[j-k]:0) )
    Then:
      E' = E + sum_k (2*C[k]*ΔC[k] + (ΔC[k])^2)
    """
    N = len(xpm1)
    xj = xpm1[j]
    dE = 0
    for k in range(1, N):
        s = 0
        jp = j + k
        jm = j - k
        if jp < N:
            s += xpm1[jp]
        if jm >= 0:
            s += xpm1[jm]
        dC = -2 * xj * s
        if dC != 0:
            dE += 2 * C[k] * dC + dC * dC
    return E + dE


def apply_flip_inplace(xpm1: list[int], C: list[int], j: int):
    """Apply flip j to xpm1 and update correlations C in-place in O(N)."""
    N = len(xpm1)
    xj = xpm1[j]
    for k in range(1, N):
        s = 0
        jp = j + k
        jm = j - k
        if jp < N:
            s += xpm1[jp]
        if jm >= 0:
            s += xpm1[jm]
        dC = -2 * xj * s
        C[k] += dC
    xpm1[j] = -xpm1[j]


# ------------------------------------------------------------
# 5) QAOA -> MTS seeding bridge
# ------------------------------------------------------------
def bitstring_to_list(bitstring: str) -> list[int]:
    return [int(b) for b in bitstring]


def get_top_k_bitstrings(counts, k_pop: int, n_bits: int) -> list[list[int]]:
    sorted_bitstrings = sorted(counts.items(), key=lambda x: -x[1])
    population = []
    for bitstring, _count in sorted_bitstrings[:k_pop]:
        padded = bitstring.zfill(n_bits)
        population.append(bitstring_to_list(padded))
    while len(population) < k_pop and len(population) > 0:
        population.append(copy.deepcopy(random.choice(population)))
    return population


def expected_labs_energy_from_counts(counts, N: int) -> float:
    shots = 0
    total = 0.0
    for bitstring, c in counts.items():
        bits = [int(b) for b in bitstring.zfill(N)]
        total += c * calculate_energy(bits, N)
        shots += c
    return total / max(shots, 1)


def schedule_params(g: float, b: float, p: int):
    gamma = [g * (i + 1) / p for i in range(p)]
    beta  = [b * (p - i) / p for i in range(p)]
    return gamma, beta


def run_qaoa_for_mts_seeding(
    n_qubits: int,
    k_pop: int,
    p_layers: int = 2,
    shots_final: int = 4096,
    optimize: bool = True,
    opt_shots: int = 256,
    opt_iters: int = 25,
    fraction_four: float = 1.0,
    fraction_two: float = 1.0,
    seed: int = 0,
):
    # Build interactions
    fb_0, fb_1, fb_2, fb_3, tb_0, tb_1 = get_labs_interactions_flat(n_qubits)

    # Optional subsampling to shrink the circuit
    fb_0, fb_1, fb_2, fb_3, tb_0, tb_1 = subsample_interactions(
        fb_0, fb_1, fb_2, fb_3, tb_0, tb_1,
        fraction_four=fraction_four,
        fraction_two=fraction_two,
        seed=seed
    )

    n_four_body = len(fb_0)
    n_two_body  = len(tb_0)

    print(f"  Problem size: {n_qubits} qubits | p={p_layers}")
    print(f"  Using 4-body terms: {n_four_body}  (fraction_four={fraction_four})")
    print(f"  Using 2-body terms: {n_two_body}   (fraction_two={fraction_two})")

    stats, total_gates, depth_serial = qaoa_gate_stats(n_qubits, p_layers, n_four_body, n_two_body)
    print(f"  Gate counts: {stats}  total={total_gates}")
    print(f"  Estimated serial depth (loop-order): {depth_serial}")

    # Optimize 2 parameters (g,b) with shot-based objective
    if optimize:
        print(f"  Optimizing (shot-based) with opt_shots={opt_shots}, iters={opt_iters} ...")

        def cost_gb(x):
            g, b = float(x[0]), float(x[1])
            gamma, beta = schedule_params(g, b, p_layers)
            counts = cudaq.sample(
                labs_qaoa_kernel,
                n_qubits, p_layers,
                gamma, beta,
                fb_0, fb_1, fb_2, fb_3,
                tb_0, tb_1,
                n_four_body, n_two_body,
                shots_count=opt_shots
            )
            return expected_labs_energy_from_counts(counts, n_qubits)

        x0 = np.array([0.3, 0.7], dtype=float)
        result = minimize(cost_gb, x0, method="COBYLA", options={"maxiter": opt_iters, "rhobeg": 0.2})
        g_opt, b_opt = float(result.x[0]), float(result.x[1])
        gamma_opt, beta_opt = schedule_params(g_opt, b_opt, p_layers)
        print(f"  Done. Estimated energy={result.fun:.3f}  g={g_opt:.3f}  b={b_opt:.3f}")
    else:
        gamma_opt, beta_opt = schedule_params(0.3, 0.7, p_layers)

    # Final sampling for population
    print(f"  Sampling {shots_final} shots for population...")
    counts = cudaq.sample(
        labs_qaoa_kernel,
        n_qubits, p_layers,
        gamma_opt, beta_opt,
        fb_0, fb_1, fb_2, fb_3,
        tb_0, tb_1,
        n_four_body, n_two_body,
        shots_count=shots_final
    )

    population = get_top_k_bitstrings(counts, k_pop, n_qubits)
    return population


# ------------------------------------------------------------
# 6) MTS (same structure, faster tabu via incremental energy)
# ------------------------------------------------------------
def combine(s1: list[int], s2: list[int], N: int):
    if N > 1:
        cut_pt = random.randint(1, N - 1)
        return s1[0:cut_pt] + s2[cut_pt:]
    return s1


def mutate(bits01: list[int], p: float, N: int):
    for i in range(N):
        if random.random() < p:
            bits01[i] ^= 1


def tabu_search_fast(N: int, bits01: list[int], max_iters=100, tabu_tenure=5):
    """
    Tabu search with incremental energy updates:
    - Maintains x in +/-1, correlations C[k], and energy E.
    - Evaluates neighbor flips in O(N) each instead of O(N^2) each.
    """
    x = bits_to_pm1(bits01)
    C = compute_correlations_pm1(x)
    E = energy_from_correlations(C)

    best_bits = bits01[:]
    best_E = E

    tabu_list = []

    for _ in range(max_iters):
        local_best_E = float("inf")
        local_best_move = None

        for j in range(N):
            neighbor_E = delta_energy_if_flip(x, C, E, j)

            is_tabu = j in tabu_list
            is_aspiration = neighbor_E < best_E

            if (not is_tabu) or is_aspiration:
                if neighbor_E < local_best_E:
                    local_best_E = neighbor_E
                    local_best_move = j

        if local_best_move is None:
            break

        # Apply chosen move
        apply_flip_inplace(x, C, local_best_move)
        E = local_best_E

        # Update best
        if E < best_E:
            best_E = E
            # convert x back to bits
            best_bits = [1 if v == 1 else 0 for v in x]

        # Tabu update
        tabu_list.append(local_best_move)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

    return best_bits


def generate_random_bitstring(k_pop, N):
    return [[random.randint(0, 1) for _ in range(N)] for _ in range(k_pop)]


def MTS(N, k_pop, num_iterations, mutation_rate=0.05, population=None):
    if population is None or len(population) == 0:
        population = generate_random_bitstring(k_pop, N)

    # Init best
    best_solution = population[0]
    best_energy = calculate_energy(best_solution, N)
    for ind in population:
        e = calculate_energy(ind, N)
        if e < best_energy:
            best_energy = e
            best_solution = copy.deepcopy(ind)

    print(f"Initial Best Energy: {best_energy}")

    for it in range(num_iterations):
        if it == 0:
            child = copy.deepcopy(random.choice(population))
        else:
            p1 = random.choice(population)
            p2 = random.choice(population)
            child = combine(p1, p2, N)

        mutate(child, mutation_rate, N)

        improved = tabu_search_fast(N, child, max_iters=N * 2, tabu_tenure=5)
        improved_E = calculate_energy(improved, N)

        if improved_E < best_energy:
            print(f"Iteration {it}: New Best Energy Found: {improved_E}")
            best_energy = improved_E
            best_solution = copy.deepcopy(improved)
            population[random.randint(0, k_pop - 1)] = improved

    return best_solution, best_energy


# ------------------------------------------------------------
# 7) Main
# ------------------------------------------------------------
if __name__ == "__main__":
    # Reproducibility
    random.seed(0)
    np.random.seed(0)

    N = 20
    k = 8
    mts_iterations = 50
    mutation_rate = 0.05

    # QAOA knobs (these matter most for speed)
    p_layers = 2
    shots_final = 4096

    # Optimization knobs
    optimize = True
    opt_shots = 256
    opt_iters = 25

    # BIG speed knob: subsample terms for seeding
    # Recommended for N=30 on a single GPU simulator:
    fraction_four = 0.25  # try 0.10 .. 0.50
    fraction_two  = 1.00  # 2-body terms are cheap; you can also subsample if needed

    print("=" * 70)
    print("HYBRID QAOA-SEEEDED MTS LABS SOLVER (SINGLE GPU OPTIMIZED)")
    print("=" * 70)

    print("\n[Step 1] Running QAOA to generate initial population...")
    initial_population = run_qaoa_for_mts_seeding(
        n_qubits=N,
        k_pop=k,
        p_layers=p_layers,
        shots_final=shots_final,
        optimize=optimize,
        opt_shots=opt_shots,
        opt_iters=opt_iters,
        fraction_four=fraction_four,
        fraction_two=fraction_two,
        seed=0
    )

    print(f"\nQuantum-seeded population ({k} individuals):")
    for idx, ind in enumerate(initial_population):
        e = calculate_energy(ind, N)
        print(f"  Individual {idx}: Energy={e}  bits={ind}")

    print(f"\n[Step 2] Running MTS with quantum-seeded population...")
    best_solution, best_energy = MTS(
        N=N,
        k_pop=k,
        num_iterations=mts_iterations,
        mutation_rate=mutation_rate,
        population=initial_population
    )

    print("\n" + "=" * 70)
    print("FINAL RESULTS")
    print("=" * 70)
    print(f"Best solution: {best_solution}")
    print(f"Best energy: {best_energy}")
    if best_energy > 0:
        print(f"Merit factor: {N**2 / (2 * best_energy):.4f}")
    else:
        print("Perfect solution found!")


HYBRID QAOA-SEEEDED MTS LABS SOLVER (SINGLE GPU OPTIMIZED)

[Step 1] Running QAOA to generate initial population...
  Problem size: 20 qubits | p=2
  Using 4-body terms: 131  (fraction_four=0.25)
  Using 2-body terms: 90   (fraction_two=1.0)
  Gate counts: {'H': 20, 'RX': 40, 'RZ': 442, 'CNOT': 1932}  total=2434
  Estimated serial depth (loop-order): 2415
  Optimizing (shot-based) with opt_shots=256, iters=25 ...
  Done. Estimated energy=178.500  g=0.305  b=0.892
  Sampling 4096 shots for population...

Quantum-seeded population (8 individuals):
  Individual 0: Energy=114  bits=[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1]
  Individual 1: Energy=130  bits=[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
  Individual 2: Energy=118  bits=[1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1]
  Individual 3: Energy=186  bits=[1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]
  Individual 4: Energy=298  bits=[0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Set to your desired GPU index

import cudaq
from cudaq import spin
import numpy as np
from scipy.optimize import minimize
import random
import copy
import time

# ============================================================
# OPTIMIZED CUDA-Q QAOA Implementation for LABS
# ============================================================

def labs_hamiltonian(n: int) -> cudaq.SpinOperator:
    """Construct the LABS cost Hamiltonian."""
    hamiltonian = 0.0 * spin.i(0)
    
    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    hamiltonian += 2.0 * spin.z(ii) * spin.z(ii + tt) * spin.z(ii + kk) * spin.z(ii + kk + tt)
    
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                hamiltonian += spin.z(ii) * spin.z(jj)
    
    return hamiltonian

def get_labs_interactions_flat(n: int):
    """Pre-compute LABS interactions as flat lists."""
    fb_0, fb_1, fb_2, fb_3 = [], [], [], []
    
    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    fb_0.append(ii)
                    fb_1.append(ii + tt)
                    fb_2.append(ii + kk)
                    fb_3.append(ii + kk + tt)
    
    tb_0, tb_1 = [], []
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                tb_0.append(ii)
                tb_1.append(jj)
    
    return fb_0, fb_1, fb_2, fb_3, tb_0, tb_1

# ============================================================
# SPEEDUP 1: Symmetry-exploiting kernel (fix first qubit to |0⟩)
# ============================================================

@cudaq.kernel
def labs_qaoa_kernel_symmetric(
    n_qubits: int,
    p_layers: int,
    gamma: list[float], 
    beta: list[float],
    fb_0: list[int], fb_1: list[int], fb_2: list[int], fb_3: list[int],
    tb_0: list[int], tb_1: list[int],
    n_four_body: int,
    n_two_body: int
):
    """
    QAOA kernel with symmetry breaking:
    - First qubit fixed to |0⟩ (breaks bit-flip symmetry, halves search space)
    """
    qubits = cudaq.qvector(n_qubits)
    
    # Initial superposition - SKIP first qubit to break bit-flip symmetry
    # Qubit 0 stays in |0⟩, others go to |+⟩
    for q_idx in range(1, n_qubits):
        h(qubits[q_idx])
    
    # Apply p QAOA layers
    for layer in range(p_layers):
        # Phase separator: 4-body ZZZZ terms
        for idx in range(n_four_body):
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            rz(4.0 * gamma[layer], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
        
        # Phase separator: 2-body ZZ terms
        for idx in range(n_two_body):
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
            rz(2.0 * gamma[layer], qubits[tb_1[idx]])
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
        
        # Mixer: apply to all qubits except first (which is fixed)
        for q_idx in range(1, n_qubits):
            rx(2.0 * beta[layer], qubits[q_idx])

# ============================================================
# SPEEDUP 2: Pre-computed/transferred parameters from the paper
# ============================================================

def get_transferred_parameters(p_layers: int, n_qubits: int) -> tuple:
    """
    Get pre-optimized parameters that transfer across problem sizes.
    Based on the paper's finding that parameters transfer well.
    
    These are approximate optimal parameters from linear ramp + refinement.
    """
    # Linear ramp schedule (works well according to the paper)
    # gamma increases, beta decreases
    gamma = []
    beta = []
    
    for layer_idx in range(p_layers):
        # Normalized layer position [0, 1]
        t = (layer_idx + 0.5) / p_layers
        
        # Empirically good schedules from QAOA literature
        # gamma: starts small, increases (phase accumulation)
        # beta: starts large, decreases (mixing strength)
        gamma.append(0.4 * t)  # Range ~[0, 0.4]
        beta.append(0.5 * (1 - t) + 0.1)  # Range ~[0.6, 0.1]
    
    return gamma, beta

def get_fourier_parameters(p_layers: int, q_coeffs: int = 2) -> tuple:
    """
    FOURIER parameterization: use only q Fourier coefficients instead of 2p parameters.
    This dramatically reduces the optimization search space.
    
    gamma_l = sum_{k=1}^{q} u_k * sin((k-0.5) * pi * l / p)
    beta_l = sum_{k=1}^{q} v_k * cos((k-0.5) * pi * l / p)
    """
    # Good default Fourier coefficients (can be optimized)
    u_coeffs = [0.3, 0.1][:q_coeffs]  # gamma coefficients
    v_coeffs = [0.4, 0.1][:q_coeffs]  # beta coefficients
    
    gamma = []
    beta = []
    
    for layer_idx in range(p_layers):
        l = layer_idx + 1
        g = sum(u_coeffs[k] * np.sin((k + 0.5) * np.pi * l / p_layers) 
                for k in range(len(u_coeffs)))
        b = sum(v_coeffs[k] * np.cos((k + 0.5) * np.pi * l / p_layers) 
                for k in range(len(v_coeffs)))
        gamma.append(g)
        beta.append(max(0.05, b))  # Keep beta positive
    
    return gamma, beta

# ============================================================
# SPEEDUP 3: Vectorized energy calculation with NumPy
# ============================================================

def calculate_energy_fast(s: np.ndarray) -> int:
    """
    Vectorized energy calculation using NumPy.
    s: 1D numpy array of 0/1 values
    """
    N = len(s)
    # Convert 0/1 to -1/+1
    seq = 2 * s - 1
    
    energy = 0
    for k in range(1, N):
        # Vectorized autocorrelation
        c_k = np.sum(seq[:N-k] * seq[k:])
        energy += c_k * c_k
    
    return int(energy)

def calculate_energy_batch(population: np.ndarray) -> np.ndarray:
    """Calculate energy for entire population at once."""
    return np.array([calculate_energy_fast(ind) for ind in population])

# ============================================================
# Canonicalization for symmetry exploitation
# ============================================================

def canonicalize_bitstring(s: list[int]) -> list[int]:
    """
    Return the canonical representative of a bitstring under LABS symmetries.
    
    Symmetries:
    1. Bit-flip: s and NOT(s) have same energy
    2. Reversal: s and REVERSE(s) have same energy
    
    Canonical form: lexicographically smallest among {s, NOT(s), REV(s), NOT(REV(s))}
    """
    s_arr = np.array(s)
    not_s = 1 - s_arr
    rev_s = s_arr[::-1]
    not_rev_s = 1 - rev_s
    
    candidates = [s_arr, not_s, rev_s, not_rev_s]
    
    # Return lexicographically smallest
    min_candidate = min(candidates, key=lambda x: tuple(x))
    return min_candidate.tolist()

def deduplicate_by_symmetry(population: list, n_bits: int) -> list:
    """
    Remove duplicates accounting for LABS symmetries.
    Returns unique canonical representatives.
    """
    seen = set()
    unique_pop = []
    
    for ind in population:
        canonical = tuple(canonicalize_bitstring(ind))
        if canonical not in seen:
            seen.add(canonical)
            unique_pop.append(list(canonical))
    
    return unique_pop

# ============================================================
# Optimized bridge functions
# ============================================================

def bitstring_to_array(bitstring: str, n_bits: int) -> np.ndarray:
    """Convert bitstring to numpy array with proper padding."""
    padded = bitstring.zfill(n_bits)
    return np.array([int(bit) for bit in padded], dtype=np.int8)

def get_top_k_bitstrings_optimized(counts, k_pop: int, n_bits: int, 
                                    use_symmetry: bool = True) -> list:
    """
    Extract top-k bitstrings, optionally exploiting symmetry.
    """
    # Sort by count
    sorted_bitstrings = sorted(counts.items(), key=lambda x: -x[1])
    
    population = []
    seen_canonical = set()
    
    for bitstring, count in sorted_bitstrings:
        arr = bitstring_to_array(bitstring, n_bits)
        ind = arr.tolist()
        
        if use_symmetry:
            canonical = tuple(canonicalize_bitstring(ind))
            if canonical not in seen_canonical:
                seen_canonical.add(canonical)
                population.append(list(canonical))
        else:
            population.append(ind)
        
        if len(population) >= k_pop:
            break
    
    # Fill if needed
    while len(population) < k_pop:
        population.append(copy.deepcopy(random.choice(population)))
    
    return population

# ============================================================
# Optimized QAOA runner
# ============================================================

def run_qaoa_for_mts_seeding_optimized(
    n_qubits: int, 
    k_pop: int, 
    p_layers: int = 3,
    shots: int = 4096, 
    optimize: bool = False,  # Default to False - use transferred params
    use_symmetry: bool = True,
    optimization_method: str = 'transferred'  # 'transferred', 'fourier', 'full'
):
    """
    Optimized QAOA runner with:
    - Symmetry exploitation
    - Transferred/Fourier parameters (faster than full optimization)
    - Efficient sampling
    """
    # Pre-compute interactions
    fb_0, fb_1, fb_2, fb_3, tb_0, tb_1 = get_labs_interactions_flat(n_qubits)
    n_four_body = len(fb_0)
    n_two_body = len(tb_0)
    
    print(f"  Problem size: {n_qubits} qubits")
    print(f"  4-body interactions: {n_four_body}")
    print(f"  2-body interactions: {n_two_body}")
    print(f"  Using symmetry breaking: {use_symmetry}")
    
    # Select kernel based on symmetry setting
    kernel = labs_qaoa_kernel_symmetric if use_symmetry else labs_qaoa_kernel_symmetric
    
    # Get parameters based on method
    if optimization_method == 'transferred':
        print(f"  Using transferred parameters (no optimization)...")
        optimal_gamma, optimal_beta = get_transferred_parameters(p_layers, n_qubits)
        
    elif optimization_method == 'fourier':
        print(f"  Using Fourier parameterization...")
        # Only optimize 4 parameters instead of 2*p
        hamiltonian = labs_hamiltonian(n_qubits)
        
        def fourier_cost(coeffs):
            u = coeffs[:2].tolist()
            v = coeffs[2:].tolist()
            
            gamma = []
            beta = []
            for l_idx in range(p_layers):
                l = l_idx + 1
                g = sum(u[k] * np.sin((k + 0.5) * np.pi * l / p_layers) for k in range(2))
                b = sum(v[k] * np.cos((k + 0.5) * np.pi * l / p_layers) for k in range(2))
                gamma.append(g)
                beta.append(max(0.05, b))
            
            return cudaq.observe(
                kernel, hamiltonian,
                n_qubits, p_layers,
                gamma, beta,
                fb_0, fb_1, fb_2, fb_3,
                tb_0, tb_1,
                n_four_body, n_two_body
            ).expectation()
        
        x0 = np.array([0.3, 0.1, 0.4, 0.1])
        result = minimize(fourier_cost, x0, method='COBYLA',
                         options={'maxiter': 50, 'rhobeg': 0.2})
        
        u = result.x[:2]
        v = result.x[2:]
        optimal_gamma = []
        optimal_beta = []
        for l_idx in range(p_layers):
            l = l_idx + 1
            g = sum(u[k] * np.sin((k + 0.5) * np.pi * l / p_layers) for k in range(2))
            b = sum(v[k] * np.cos((k + 0.5) * np.pi * l / p_layers) for k in range(2))
            optimal_gamma.append(g)
            optimal_beta.append(max(0.05, b))
        print(f"  Fourier optimization complete. Energy: {result.fun:.4f}")
        
    else:  # Full optimization
        print(f"  Running full parameter optimization...")
        hamiltonian = labs_hamiltonian(n_qubits)
        
        def cost_function(params):
            gamma_params = params[:p_layers].tolist()
            beta_params = params[p_layers:].tolist()
            return cudaq.observe(
                kernel, hamiltonian,
                n_qubits, p_layers,
                gamma_params, beta_params,
                fb_0, fb_1, fb_2, fb_3,
                tb_0, tb_1,
                n_four_body, n_two_body
            ).expectation()
        
        x0 = np.array([0.1 * (idx + 1) / p_layers for idx in range(p_layers)] +
                      [0.5 * (p_layers - idx) / p_layers for idx in range(p_layers)])
        
        result = minimize(cost_function, x0, method='COBYLA',
                         options={'maxiter': 100, 'rhobeg': 0.3})
        
        optimal_gamma = result.x[:p_layers].tolist()
        optimal_beta = result.x[p_layers:].tolist()
        print(f"  Full optimization complete. Energy: {result.fun:.4f}")
    
    # Sample from circuit
    print(f"  Sampling {shots} shots...")
    counts = cudaq.sample(
        kernel,
        n_qubits, p_layers,
        optimal_gamma, optimal_beta,
        fb_0, fb_1, fb_2, fb_3,
        tb_0, tb_1,
        n_four_body, n_two_body,
        shots_count=shots
    )
    
    # Convert to MTS format with symmetry deduplication
    population = get_top_k_bitstrings_optimized(counts, k_pop, n_qubits, use_symmetry)
    
    return population

# ============================================================
# Optimized MTS with incremental energy updates
# ============================================================

def calculate_energy_delta(s: np.ndarray, flip_idx: int, current_energy: int) -> int:
    """
    Calculate new energy after flipping bit at flip_idx.
    Uses incremental update - much faster than full recalculation.
    """
    N = len(s)
    seq = 2 * s - 1  # Convert to +/- 1
    
    # Flipping bit i affects all autocorrelations C_k where:
    # - i appears in the sum (i.e., i < N-k), contributing s_i * s_{i+k}
    # - i+k appears in the sum (i.e., i >= k), contributing s_{i-k} * s_i
    
    old_contrib = 0
    new_contrib = 0
    
    for k in range(1, N):
        c_k_old = np.sum(seq[:N-k] * seq[k:])
        
        # Delta for this k
        delta_c_k = 0
        
        # Term where flip_idx is the first index: s[flip_idx] * s[flip_idx + k]
        if flip_idx + k < N:
            delta_c_k -= 2 * seq[flip_idx] * seq[flip_idx + k]
        
        # Term where flip_idx is the second index: s[flip_idx - k] * s[flip_idx]
        if flip_idx - k >= 0:
            delta_c_k -= 2 * seq[flip_idx - k] * seq[flip_idx]
        
        c_k_new = c_k_old + delta_c_k
        old_contrib += c_k_old * c_k_old
        new_contrib += c_k_new * c_k_new
    
    return current_energy + (new_contrib - old_contrib)

def tabu_search_fast(N: int, s: list[int], max_iters: int = 100, tabu_tenure: int = 5):
    """
    Optimized Tabu Search with:
    - NumPy arrays instead of lists
    - Set for O(1) tabu lookup
    - Incremental energy updates (optional, more complex)
    """
    current_s = np.array(s, dtype=np.int8)
    best_s = current_s.copy()
    best_energy = calculate_energy_fast(current_s)
    current_energy = best_energy
    
    tabu_set = set()  # O(1) lookup instead of O(n)
    tabu_queue = []   # To maintain order for removal
    
    for _ in range(max_iters):
        local_best_idx = -1
        local_best_energy = float('inf')
        
        for i_val in range(N):
            # Flip bit temporarily
            current_s[i_val] ^= 1
            neighbor_energy = calculate_energy_fast(current_s)
            current_s[i_val] ^= 1  # Flip back
            
            is_tabu = i_val in tabu_set
            is_aspiration = neighbor_energy < best_energy
            
            if (not is_tabu) or is_aspiration:
                if neighbor_energy < local_best_energy:
                    local_best_energy = neighbor_energy
                    local_best_idx = i_val
        
        if local_best_idx >= 0:
            current_s[local_best_idx] ^= 1
            current_energy = local_best_energy
            
            if local_best_energy < best_energy:
                best_energy = local_best_energy
                best_s = current_s.copy()
            
            # Update tabu structure
            tabu_set.add(local_best_idx)
            tabu_queue.append(local_best_idx)
            if len(tabu_queue) > tabu_tenure:
                old_idx = tabu_queue.pop(0)
                tabu_set.discard(old_idx)
    
    return best_s.tolist()

def MTS_fast(N: int, k_pop: int, num_iterations: int, 
             mutation_rate: float = 0.05, population: list = None):
    """
    Optimized MTS with NumPy operations.
    """
    if population is None or len(population) == 0:
        population = [np.random.randint(0, 2, N, dtype=np.int8).tolist() 
                      for _ in range(k_pop)]
    
    # Convert to numpy for faster operations
    pop_array = np.array(population, dtype=np.int8)
    
    # Find initial best
    energies = calculate_energy_batch(pop_array)
    best_idx = np.argmin(energies)
    best_solution = pop_array[best_idx].copy()
    best_energy = energies[best_idx]

    print(f"Initial Best Energy: {best_energy}")

    for it in range(num_iterations):
        if it == 0:
            child = pop_array[random.randint(0, k_pop-1)].copy()
        else:
            p1_idx = random.randint(0, k_pop-1)
            p2_idx = random.randint(0, k_pop-1)
            cut_pt = random.randint(1, N-1)
            child = np.concatenate([pop_array[p1_idx, :cut_pt], 
                                   pop_array[p2_idx, cut_pt:]])
        
        # Mutate using vectorized operation
        mutation_mask = np.random.random(N) < mutation_rate
        child = child ^ mutation_mask.astype(np.int8)
        
        # Tabu search
        improved_child = tabu_search_fast(N, child.tolist(), max_iters=N*2)
        improved_energy = calculate_energy_fast(np.array(improved_child))

        if improved_energy < best_energy:
            print(f"Iteration {it}: New Best Energy Found: {improved_energy}")
            best_energy = improved_energy
            best_solution = np.array(improved_child, dtype=np.int8)
            
            replace_idx = random.randint(0, k_pop - 1)
            pop_array[replace_idx] = best_solution

    return best_solution.tolist(), int(best_energy)

# ============================================================
# Main: Optimized Hybrid Quantum-Classical LABS Solver
# ============================================================

if __name__ == "__main__":

    start = time.time()
    
    N = 31
    k = 15
    mts_iterations = 50
    mutation_rate = 0.05

    print("=" * 60)
    print("OPTIMIZED HYBRID QUANTUM-CLASSICAL LABS SOLVER")
    print("=" * 60)
    print(f"\nOptimizations enabled:")
    print(f"  - Symmetry breaking (halves quantum search space)")
    print(f"  - Transferred parameters (skips expensive optimization)")
    print(f"  - NumPy vectorization (faster energy calculations)")
    print(f"  - Set-based tabu list (O(1) lookup)")

    # Step 1: Run optimized QAOA
    print("\n[Step 1] Running QAOA to generate initial population...")
    initial_population = run_qaoa_for_mts_seeding_optimized(
        n_qubits=N, 
        k_pop=k, 
        p_layers=3,  # Can use more layers since we're not optimizing
        shots=4096,
        use_symmetry=True,
        optimization_method='transferred'  # 'transferred', 'fourier', or 'full'
    )

    print(f"\nQuantum-seeded population ({k} individuals):")
    for idx, ind in enumerate(initial_population):
        energy = calculate_energy_fast(np.array(ind))
        print(f"  Individual {idx}: {ind} -> Energy: {energy}")

    qaoa_time = time.time() - start
    print(f"\nQAOA phase completed in {qaoa_time:.2f} seconds")

    # Step 2: Run optimized MTS
    print(f"\n[Step 2] Running MTS with quantum-seeded population...")
    mts_start = time.time()
    best_solution, best_energy = MTS_fast(
        N=N, 
        k_pop=k, 
        num_iterations=mts_iterations, 
        mutation_rate=mutation_rate, 
        population=initial_population
    )
    mts_time = time.time() - mts_start

    print("\n" + "=" * 60)
    print("FINAL RESULTS")
    print("=" * 60)
    print(f"Best solution: {best_solution}")
    print(f"Best energy: {best_energy}")
    if best_energy > 0:
        print(f"Merit factor: {N**2 / (2 * best_energy):.4f}")
    else:
        print("Perfect solution found!")

    total_time = time.time() - start
    print(f"\n[Timing]")
    print(f"  QAOA phase: {qaoa_time:.2f}s")
    print(f"  MTS phase:  {mts_time:.2f}s")
    print(f"  Total:      {total_time:.2f}s")


: 